In [17]:
import numpy as np 
import pandas as pd
import re

In [2]:
df=pd.read_csv('IMDB Dataset.csv')

In [3]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [4]:
def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean,'',text)
    

In [5]:
def convert_low(text):
    return text.lower()

In [6]:
def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [7]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [8]:
def remove_stopper(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y

In [9]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [10]:
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [11]:
def join_back(list_input):
    return " ".join(list_input)

In [12]:
df['review']=df['review'].apply(clean_html)
df['review']=df['review'].apply(convert_low)
df['review']=df['review'].apply(remove_special)
df['review']=df['review'].apply(remove_stopper)
df['review']=df['review'].apply(stem_words)
df['review']=df['review'].apply(join_back)

In [13]:
df.head()

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Split the data into training and test sets
X_train, X_val, Y_train, Y_val = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42,stratify=df.sentiment)

# print(Y_train.shape)

# print(Y_val.value_counts())

vectorizer = TfidfVectorizer(max_features=10)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

#print(X_val.iloc[0])


def knn_predict(test_vector, k=5):
    distances = cdist(X_val_tfidf.toarray(), test_vector.toarray(), metric='euclidean')
    #print(distances)
    indices = distances.argsort(axis=0)[:k]
    # print(indices)
    #print([df.iloc[i]['sentiment'] for i in indices])
    neighbors = Y_val.iloc[indices.flatten()]
    return neighbors.mode().values[0]

Y_knn_result = []
#print(len(Y_val))
#print(len(X_val))
# print(len(df))
for i in range(len(X_val)):
    test_vector = vectorizer.transform([X_val.iloc[i]])
    prediction = knn_predict(test_vector)
    Y_knn_result.append(prediction)
# print(len(Y_knn_result))

print("KNN Accuracy:", accuracy_score(Y_val, Y_knn_result))
#s=vectorizer.transform(["good movie,love it ,good,good,good, awesome,beautiful"])
# print(s)
# print(knn_predict(s))

KNN Accuracy: 0.6834
